![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2F01+-+Data+Sources&dt=01+-+BigQuery+-+Table+Data+Source.ipynb)

# 01 - BigQuery - Table Data Source
Use BigQuery to load and prepare data for machine learning:

**Video Walkthrough of this notebook:**

Includes conversational walkthrough and more explanatory information than the notebook:
<p align="center" width="100%"><center><a href="https://youtu.be/Z5whg20WvS8" target="_blank" rel="noopener noreferrer"><img src="../architectures/thumbnails/playbutton/01.png" width="40%"></a></center></p>

**Prerequisites:**
-  [00 - Environment Setup](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb)

**Resources:**
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Download BigQuery Data to Pandas](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)
-  [BigQuery Template Notebooks](https://github.com/GoogleCloudPlatform/bigquery-notebooks/tree/main/notebooks/official/template_notebooks)
- Using BigQuery From Python, Notebooks in This Repository:
    - [Tips/BigQuery - Python Client](../Tips/BigQuery%20-%20Python%20Client.ipynb)
    - [03 - BigQuery ML (BQML)/Introduction to BigQuery ML (BQML)](../03%20-%20BigQuery%20ML%20(BQML)/Introduction%20to%20BigQuery%20ML%20(BQML).ipynb)
    - [Applied Forecasting/1 - BigQuery Time Series Forecasting Data Review and Preparation](../Applied%20Forecasting/1%20-%20BigQuery%20Time%20Series%20Forecasting%20Data%20Review%20and%20Preparation.ipynb)

**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/01_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/01_console.png" width="45%">
</p>

---
## Source Data

**Overview**

This notebook imports source data for this project into Google BigQuery.  All the remaining notebooks utilize BigQuery as the source and leverage API's native to the machine learning approaches they feature.

This notebook, `01 - BigQuery - Table Data Source`, starts the machine learning lifecycle by exporting a BigQuery source table to CSV format in a Cloud Storage Bucket, then importing source data and preparing it for machine learning.

All of these workflows utilize tabular data to fit a supervised learning model: predict a target variable by learning patterns in feature columns.  The type of supervised learning used in these projects is classification: models with a target variable that has multiple discrete classes.  

**The Data**

The source data is first exported to Google Cloud Storage in CSV format below.  The BigQuery source table is `bigquery-public-data.ml_datasets.ulb_fraud_detection`.  This is a table of credit card transactions that are classified as fradulant, `Class = 1`, or normal `Class = 0`.    
- The data can be researched further at this [Kaggle link](https://www.kaggle.com/mlg-ulb/creditcardfraud).
- Read mode about BigQuery public datasets [here](https://cloud.google.com/bigquery/public-data)

**Description of the Data**

This is a table of 284,807 credit card transactions classified as fradulant or normal in the column `Class`.  In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction
>**Quick Note on PCA**<p>PCA is an unsupervised learning technique: there is not a target variable.  PCA is commonlly used as a variable/feature reduction technique.  If you have 100 features then you could reduce it to a number p (say 10) projected features.  The choice of this number is a balance of how well it can explain the variance of the full feature space and reducing the number of features.  Each projected feature is orthogonal to each other feature, meaning there is no correlation between these new projected features.</p>

**Preparation of the Data**

This notebook adds two columns to the source data and stores it in a new table with suffix `_prepped`.  
- `transaction_id` (string) a unique id for the row/transaction
- `splits` (string) this divided the tranactions into sets for `TRAIN` (80%), `VALIDATE` (10%), and `TEST` (10%)

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'avid-streamer-396319'

In [2]:
REGION = 'us-central1'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

packages:

In [3]:
from google.cloud import bigquery
from google.cloud import storage

Create clients and authenticated to our project:

In [6]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

parameters:

In [7]:
BUCKET = PROJECT_ID

---
## Store the Source Data in GCS Storage Bucket
Check to see if the export exist and create if not:
- export from bigquery table to GCS bucket as CSV
    - the table is referenced in the `BQ_SOURCE` variable at the top of this notebook
- [Exporting Table Data](https://cloud.google.com/bigquery/docs/exporting-data#python)
- [BigQuery Python Client](https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_extract_table)

For more tips on interacting with GCS using the Python Client review the tips notebook [Python Client for GCS](../Tips/Python%20Client%20for%20GCS.ipynb).

In [8]:
file = f"{SERIES}/{EXPERIMENT}/data/{BQ_TABLE}.csv"

In [9]:
bucketDef = gcs.bucket(BUCKET)
#check if the file exists in the bucket
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print(f'The file has already been created at: gs://{bucketDef.name}/{file}')
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = [f'gs://{bucketDef.name}/{file}'])
    print('Creating the export ...')
    extract.result()
    print(f'Exported the table to: gs://{bucketDef.name}/{file}')

Creating the export ...
Exported the table to: gs://avid-streamer-396319/01/01/data/fraud.csv


list files in the bucket:

In [10]:
list(bucketDef.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}'))

[<Blob: avid-streamer-396319, 01/01/data/fraud.csv, 1692640692335676>]

In [11]:
print(f'Review the files in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES};tab=objects&project={PROJECT_ID}')

Review the files in the console here:
https://console.cloud.google.com/storage/browser/avid-streamer-396319/01;tab=objects&project=avid-streamer-396319


---
## Create BigQuery Dataset

We have Projects => Buckets (datasets that organize infos) => assets (table: rows/cols)

List BigQuery datasets in the project:

In [13]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

Create the dataset if missing:

In [14]:
ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = REGION
ds.labels = {'experiment': f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

List BigQuery datasets in the project:

In [15]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

fraud


go to BigQuery => check the fraud dataset under the project name

---
## Create BigQuery Table inside Fraud dataset
- import data from Cloud Storage Bucket
- [Loading CSV data from Cloud Storage](https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv)
- [BigQuery Python Client](https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_extract_table)

In [16]:
from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
except NotFound as error:
    print(f'Creating Table ...')
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True, #detect cols types auto
        labels = {'experiment': f'{EXPERIMENT}'}
    )
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print(f'Finished creating table: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

Creating Table ...
Finished creating table: avid-streamer-396319.fraud.fraud


### Retrieve and Review a Sample From The Table:
> **Note:** The `LIMIT 5` statement does limit the number of rows returned by BigQuery to 5 but BigQuery still does a full table scan.  If you have a table larger than 1GB and want to limit the rows scanned for a quick review like then then replacing `LIMIT 5` with `TABLESAMPLE SYSTEM (1 PERCENT)` would be more efficient.  For tables under 1GB it will still return the full table.  More on [Table Sampling](https://cloud.google.com/bigquery/docs/table-sampling)

In [18]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,150336,2.030851,-0.235559,-2.804940,-0.547376,2.441730,3.277629,-0.652959,0.843118,0.542395,...,-0.058377,-0.092950,0.203618,0.642094,-0.105975,0.502090,-0.024706,-0.040995,0.78,0
1,84068,-0.679466,1.334849,1.367987,0.856085,0.000189,-0.701735,0.582134,-0.003885,-0.741823,...,0.077499,0.311384,-0.095417,0.413387,-0.212580,-0.457638,-0.250283,-0.139162,0.78,0
2,122091,2.141596,0.077735,-2.104923,0.061232,0.368903,-1.825884,0.748172,-0.564815,0.354072,...,0.131269,0.490071,-0.035338,0.008649,0.415350,0.245709,-0.091809,-0.081979,0.78,0
3,136167,2.073311,0.265580,-1.745544,0.492756,0.286109,-1.427709,0.310907,-0.422277,0.505541,...,0.191552,0.767662,-0.043917,-0.156863,0.295408,-0.096150,0.002101,-0.031715,0.78,0
4,150523,2.032967,-0.387542,-0.505876,0.350922,-0.424833,-0.134032,-0.623411,-0.005357,1.481508,...,0.151034,0.737664,0.044700,-0.682962,-0.007459,-0.166488,0.047286,-0.044997,0.78,0


### Check out this table in BigQuery Console:
- Click: https://console.cloud.google.com/bigquery
- Make sure project selected is the one from this notebook
- Under Explore, expand this project and review the dataset and table

In [19]:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")

Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=avid-streamer-396319


---
## Review Data in BigQuery
Additional SQL queries could be used to review the data.  This section shows moving the table to a Pandas dataframe for local review in Python:

> **Note:** <p>This query only selects one column.  This means BigQuery scans less data as it does not process the other columns.  </p>

In [22]:
query = f"""
SELECT Class
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
df = bq.query(query = query).to_dataframe()

In [23]:
df['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: Int64

In [24]:
df['Class'].value_counts(normalize=True)

Class
0    0.998273
1    0.001727
Name: proportion, dtype: Float64

---
## Prepare Data for Analysis

Create a prepped version of the data with test/train splits using SQL DDL: we will create new table from the existing table.

- FARM_FINGERPRINT: generates big integer => /10 and keep remainder = [0-9] => 80% as train, 10% for test and valid
- We use this splitting approach to allow every user to have the same sets for train/ test/valid

In [25]:
query = f"""
CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`) #create unique id for each row / add_id is a dummy table
SELECT *,
    CASE
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [26]:
(job.ended-job.started).total_seconds()

8.48

In [27]:
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

70.632136 MB


Review the test/train split:

In [28]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,VALIDATE,28630,10.052421
1,TEST,28310,9.940065
2,TRAIN,227867,80.007514


Retrieve a subset of the data to a Pandas dataframe:

In [29]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [30]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,119592,2.139741,0.245651,-2.654856,0.178287,1.336991,-0.724664,0.906032,-0.436125,-0.528015,...,-0.216033,0.345316,0.747103,0.700184,-0.123739,-0.099989,0.0,0,c1908dfc-b9fe-4329-947f-ce1d8b705ac3,TEST
1,129420,-0.539919,1.273124,0.987345,2.207887,0.827857,1.099094,0.154173,0.567619,-1.140350,...,-0.039536,0.308004,-1.128751,-0.291094,-0.157021,-0.031830,0.0,0,f97a4ab2-dfa3-4745-bfb5-19796e93f0c1,TEST
2,75176,1.235603,0.041383,0.675286,0.836279,-0.675016,-0.657342,-0.154209,-0.067491,0.602617,...,0.088164,0.396205,0.324557,0.182930,-0.017115,0.014979,0.0,0,41e4cf07-f271-45fe-babe-88af8c4a9d69,TEST
3,167575,1.808792,-0.632766,-0.547739,0.142396,0.222408,1.824899,-1.068149,0.721798,0.914282,...,0.262650,-1.741824,-0.606627,0.493312,0.049780,-0.075780,0.0,0,ec304a05-c031-43ab-a903-461cd394de99,TEST
4,2891,-1.356410,0.434828,3.176296,2.542596,0.250651,0.471825,-0.042386,0.066513,-0.451823,...,0.019940,0.423206,0.036699,0.059171,-0.253716,-0.105177,0.0,0,5f3e52ef-8e16-474b-a2c1-40a2f4ddac4d,TEST


---
## Remove Resources
see notebook "99 - Cleanup"

### Questions

##### Do we need to use BigQuery for data source ?
no we can use any data source. However, BigQuery is very easy to share and collaborate. It makes it easier to create same data splits for eveyone. It gives usoptions when we want to plug in ML algorithims to a source that might not be able to hold all the data. BigQuery fetches data batches in parallel => local memory doesnt need to be big enough